<h1>Open Street Map - Goiânia GO (Brazil)</h1>

<h2>Map Area </h2>

<p>The choosen area is the city that I live, Goiânia state of Goiás in Brazil.</p>

[https://www.openstreetmap.org/relation/334547](https://www.openstreetmap.org/relation/334547)


I choose Goiânia because I live here and I think I can validate the information properly.

<h2> Problems in the Map </h2>

I found some addresses with street attribute but missing street names, some misspelling in the names, and some abreviations. I also found some wrong postal codes that are not supposed to be from Goiânia.

<h3>Street Names</h3>

In Brazil usually the addresses have three main parts, street, block and lot. Some addresses have a number to, but if you have street, block and lot the number is not necessary.

I found 29 addresses with just the block and lot information. There is no information about the neighborhood. It complicate even more the identification of the addresses. Two different users did this mistakes.

In [ ]:
result = db.goiania.find({"addr.street": {'$regex': '^Qd', '$options': 'i'}})
print db.goiania.find({"addr.street": {'$regex': '^Qd', '$options': 'i'}}).count()
print db.goiania.find({"addr.street": {'$regex': '^Qd', '$options': 'i'}}).distinct("created.user")
for doc in result:
    print pprint.pprint(doc)

#result     
29

[u'MGMorais', u'erickdeoliveiraleal']

{u'_id': ObjectId('5ab983cd659c2a16ec184ec9'),
 u'addr': {u'street': u'Qd20 Lt04'},
 u'caract': {u'historic': u'castle',
             u'name': u'Carla TER\xc7A SEXTA',
             u'opening_hours': u'Tu, Fr 07:30-16:00'},
 u'created': {u'changeset': u'50281768',
              u'timestamp': u'2017-07-14T12:20:05Z',
              u'uid': u'6265328',
              u'user': u'MGMorais',
              u'version': u'3'},
 u'id': u'4953787721',
 u'pos': [-16.746967, -49.1752213],
 u'type': u'node'}    

In Brazil we have some different names to designate a street. I used the function below (audit_street_type) to fullfill the mapping variable with the values that could be replaced.

In [ ]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
#Mapping pos audit type
mapping = { "R": "Rua",
            "R.": "Rua",
            "r." : "Rua",
            "rua": "Rua",
            "RUA": "Rua",
            "Rua-": "Rua ",
            "Ruas": "Rua",
            "Av.": "Avenida",
            "AV.": "Avenida",
            "AVENIDA":"Avenida",
            "Alamenda": "Alameda",
            "Rodovoa": "Rodovia"
            }

<h3>Postal codes</h3>

I look for all the documents that have addr and postal code attributes. Only 3000 documents from 319027 have the addr attribute. Among all the addr available only 167 documents have the postal code attribute, and 12 of then are not from Goiânia area. The pattern to validate a postal code is:
* Starts with 74 (Goiania area)
* 8 digits formatted or not
* After the second digit can be a "."
* After the fifth digit can be a "-"

In [ ]:
#count quantity addr tags
print db.goiania.find({"addr": {'$exists': True}}).count()  

#count quantity postal codes 
print db.goiania.find({"addr.postcode":{'$exists':True}}).count()

#count quantity of wrong postal codes
print db.goiania.find({"addr.postcode":{'$exists':True, "$not": re.compile(r"^74\.?\d{3}\-?\d{3}$")}}).count()

#city addr.city for wrong postal codes
print db.goiania.find({"addr.postcode":{'$exists':True, "$not": re.compile(r"^74\.?\d{3}\-?\d{3}$")}}).distinct("addr.city")

#result
3246
167
12
{u'_id': ObjectId('5ab983ca659c2a16ec174d6b'),
 u'addr': {u'city': u'Goianira', u'postcode': u'75370000'},
 u'caract': {u'name': u'Escola Municipal Cora Coralina',
             u'office': u'educational_institution'},
 u'created': {u'changeset': u'26921103',
              u'timestamp': u'2014-11-21T03:23:19Z',
              u'uid': u'2469878',
              u'user': u'jwilliansilva',
              u'version': u'1'},
 u'id': u'3198044068',
 u'pos': [-16.5393776, -49.4020543],
 u'type': u'node'}
#cities
[u'Goianira', u'Santo Ant\xf4nio de Goi\xe1s', u'Trindade', u'Santo Antonio de Goi\xe1s']

The cities found in the problematic postal codes are nearby, but should not be within Goiânia boundaries.

<h2>Overview of the data</h2>

This section contains basic statistics about the dataset, the MongoDB queries used to gather them, and some additional ideas about the data in context.

<b>File Sizes</b>

map.xml...............63,1 MB<br>
map.xml.json........93,9 MB<br>


<h3>Number of unique users</h3>



385 different users contributed to the data.

In [ ]:
user = db.goiania.distinct("created.user")

print len(user)

385

82 users made only one contribution.

In [ ]:
#unique contributors
query = []

query.append({"$group" : {"_id" : "$created.user", "count":{"$sum":1}}})
query.append({"$match" : {"count" : 1}})

result = db.goiania.aggregate(query)
print len(list(db.goiania.aggregate(query))) 

for doc in result:
    pprint.pprint(doc)

#result    
82
{u'_id': u'Douglas Soares Pereira', u'count': 1}
{u'_id': u'marthaleena', u'count': 1}
{u'_id': u"DonomarBarber'Shop", u'count': 1}
{u'_id': u'wambacher', u'count': 1}
{u'_id': u'Paulo Henrique Rodrigues Oliveira', u'count': 1}
{u'_id': u'Coyoty', u'count': 1}
{u'_id': u'marciocr', u'count': 1}    

<h3> Top 10 contributors </h3>

In [ ]:
#quantidade de contribuições por usuario
query = []
query.append({"$group" : {"_id" : "$created.user", "count":{"$sum":1}}})
query.append({"$sort" : {"count": -1}})
query.append({"$limit" : 10})

result = db.goiania.aggregate(query)

for doc in result:
    pprint.pprint(doc)
    
{u'_id': u'street0501', u'count': 105016}
{u'_id': u'greecemapper', u'count': 33594}
{u'_id': u'geocorreiosgo', u'count': 32420}
{u'_id': u'erickdeoliveiraleal', u'count': 32363}
{u'_id': u'jeffersonbr', u'count': 13613}
{u'_id': u'T\xfallio', u'count': 10010}
{u'_id': u'Vilmar Amaral', u'count': 9005}
{u'_id': u'taciofernandes', u'count': 8136}
{u'_id': u'Dangoh', u'count': 6533}
{u'_id': u'jdaraujo', u'count': 6172}

<h3> Quantity of documents by type </h3>

In [ ]:
#quantity by type
query = []
query.append({"$group" : {"_id" : "$type", "count":{"$sum":1}}})
query.append({"$sort" : {"count": -1}})

result = db.goiania.aggregate(query)

for doc in result:
    pprint.pprint(doc)
    
{u'_id': u'node', u'count': 267749}
{u'_id': u'way', u'count': 51278}

<h3> Top 10 amenities </h3>

In [ ]:
#quantidade de registro por tipo 
query = []
query.append({"$group" : {"_id" : "$caract.amenity", "count":{"$sum":1}}})
query.append({"$sort" : {"count": -1}})
query.append({"$limit" : 10})

result = db.goiania.aggregate(query)

for doc in result:
    pprint.pprint(doc)
    
{u'_id': None, u'count': 317771}
{u'_id': u'parking', u'count': 165}
{u'_id': u'fuel', u'count': 160}
{u'_id': u'bank', u'count': 112}
{u'_id': u'school', u'count': 98}
{u'_id': u'place_of_worship', u'count': 88}
{u'_id': u'fast_food', u'count': 80}
{u'_id': u'restaurant', u'count': 78}
{u'_id': u'pharmacy', u'count': 76}
{u'_id': u'post_office', u'count': 54}    

Parking is the most common amenity. 

Most part of the restaurants are without the cuisine information. For the ones with this information, the most popular cuisine is Pizza.

In [ ]:
#amenity: restaurant cousine
print db.goiania.find({"caract.amenity":{"$exists": True, "$eq": u'restaurant'}}).distinct("caract.cuisine")
#most frequent cuisine 
query = []
query.append({"$match":{"caract.amenity":{"$exists": True, "$eq": u'restaurant'}}})
query.append({"$group" : {"_id" : "$caract.cuisine", "count":{"$sum":1}}})
query.append({"$sort" : {"count": -1}})
cuisine = db.goiania.aggregate(query)
for doc in cuisine:
    print pprint.pprint(doc)
#result
[u'barbecue', u'japanese', u'pizza', u'steak_house;burger', u'chinese', u'pasta', u'regional', u'italian', 
 u'international', u'burger', u'barbecue;steak_house;brazilian', u'vegetarian', u'barbecue;japanese', u'chinese;oriental', 
 u'steak_house']

{u'_id': None, u'count': 47}
{u'_id': u'pizza', u'count': 6}
{u'_id': u'regional', u'count': 5}
{u'_id': u'burger', u'count': 3}
{u'_id': u'japanese', u'count': 3}
{u'_id': u'barbecue', u'count': 3}
{u'_id': u'international', u'count': 2}
{u'_id': u'chinese;oriental', u'count': 1}
{u'_id': u'barbecue;japanese', u'count': 1}
{u'_id': u'barbecue;steak_house;brazilian', u'count': 1}
{u'_id': u'vegetarian', u'count': 1}
{u'_id': u'steak_house;burger', u'count': 1}
{u'_id': u'pasta', u'count': 1}
{u'_id': u'steak_house', u'count': 1}
{u'_id': u'chinese', u'count': 1}
{u'_id': u'italian', u'count': 1}


Another data about food that are present among the amenities are fast_food. I searched about the fast food cuisine also.

In [ ]:
#amenity: restaurant cousine
print db.goiania.find({"caract.amenity":{"$exists": True, "$eq": u'fast_food'}}).distinct("caract.cuisine")
#most frequent cuisine 
query = []
query.append({"$match":{"caract.amenity":{"$exists": True, "$eq": u'fast_food'}}})
query.append({"$group" : {"_id" : "$caract.cuisine", "count":{"$sum":1}}})
query.append({"$sort" : {"count": -1}})
cuisine = db.goiania.aggregate(query)
for doc in cuisine:
    print pprint.pprint(doc)

#result    
[u'burger', u'sandwich', u'burger;arab', u'burger;sandwich', u'pastel;salgados;coca_gelada;sucos;simpatia', u'sandwiches', u'regional;ice_cream', u'pizza;burger']
{u'_id': None, u'count': 65}
{u'_id': u'burger', u'count': 6}
{u'_id': u'sandwich', u'count': 3}
{u'_id': u'regional;ice_cream', u'count': 1}
{u'_id': u'sandwiches', u'count': 1}
{u'_id': u'burger;arab', u'count': 1}
{u'_id': u'pastel;salgados;coca_gelada;sucos;simpatia', u'count': 1}
{u'_id': u'burger;sandwich', u'count': 1}
{u'_id': u'pizza;burger', u'count': 1}

Since religion plays a big role in Brazilian culture I will look for the biggest religion. My guess is that is christian.

In [ ]:
#amenity: place_of_worship religion
print db.goiania.find({"caract.amenity":{"$exists": True, "$eq": u'place_of_worship'}}).distinct("caract.religion")
#most frequent religion 
query = []
query.append({"$match":{"caract.amenity":{"$exists": True, "$eq": u'place_of_worship'}}})
query.append({"$group" : {"_id" : "$caract.religion", "count":{"$sum":1}}})
query.append({"$sort" : {"count": -1}})
religion = db.goiania.aggregate(query)
for doc in religion:
    print pprint.pprint(doc)
    
[u'christian', u'Spiritist']
{u'_id': u'christian', u'count': 66}

{u'_id': None, u'count': 21}

{u'_id': u'Spiritist', u'count': 1}


<h2> Additional information </h2>

Some problems that I found analizing this data and suggestions to improvements.
<br>
<h3> Lack of addresses annotations</h3>



The data needs more addresses annotations. Parking is the amenity with more annotations (165) but few of then (3) have addresses information. Other amenities have the same problem.     

In [ ]:
#information about the most common amenity parking
print db.goiania.find({"caract.amenity":{"$exists": True, "$eq": u'parking'}}).count()
print db.goiania.find({"$and" : [{"caract.amenity":{"$exists": True, "$eq": u'parking'}}, {"addr":{"$exists": True}}]}).count()
#result
165
3

#information about quantity of addr in the amenities restaurant
print db.goiania.find({"caract.amenity":{"$exists": True, "$eq": u'restaurant'}}).count()
print db.goiania.find({"$and" : [{"caract.amenity":{"$exists": True, "$eq": u'restaurant'}}, {"addr":{"$exists": True}}]}).count()
#result
78
14

#information about quantity of addr in the amenities fuel
print db.goiania.find({"caract.amenity":{"$exists": True, "$eq": u'fuel'}}).count()
print db.goiania.find({"$and" : [{"caract.amenity":{"$exists": True, "$eq": u'fuel'}}, {"addr":{"$exists": True}}]}).count()
#result
160
12

#information about quantity of addr in the amenities bank
print db.goiania.find({"caract.amenity":{"$exists": True, "$eq": u'bank'}}).count()
print db.goiania.find({"$and" : [{"caract.amenity":{"$exists": True, "$eq": u'bank'}}, {"addr":{"$exists": True}}]}).count()
#result
112
11

1. Solution.<br>
> We can use the Brasilian post office database to annotate more data with their addresses information.<br>
    
* Benefits<br>
> Complete information. It's not enough to know the latitude and longitude. The addresses is a more friendly information for the end user.<br>
* Antecipated Issues<br>
> Brasilian post office company (Correios) sells this database, and there is latitute and longitude information as long as addresses. If buying is not an option, some old versions are available in the internet but probably not updated like if we buy from Correios.

<h3> Few contributions </h3>
<br> 
<br>
Goiânia map needs more contributions.

1. Solution.<br>
> Open Street Map initiative should try to contact and advertise in public universities in Brazil. The professors could use projects like this to evaluate the students, and this could contribute to the project. 
    
* Benefits<br>
> Increase in data quantity.<br>
* Antecipated Issues<br>
> We can't be sure about the quality of data provided. The professors could develop a methodology to evaluate an try to prevent bad quality data.


<br>
<br>

Gamefication is also a good strategy to maintain the contributors interested. 

1. Solution.<br>
> Provide an interesting gamification strategy to contributors. 
    
* Benefits<br>
> Increases the number of contributors and maintains the interest in regular contribution.<br>
* Antecipated Issues<br>
> A regular developer team is necessary to make and maintain the changes. Maintaining a private team could increase the costs of the project. Counting with voluntary contribution may take longer than we expect.
